# 5) Feature Selection & Engineering
Analyse der Korrelationen und Kausalitäten, Identifikation der Features (erklärende Variablen),
ggf. Optimierung der Datengrundlage

## 5.1 Importieren der Bibliotheken & drive mount

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the folder paths
main_folder_path = '/content/drive/MyDrive/wine_project'
processed_data_folder_path = os.path.join(main_folder_path, 'processed_data')
training_data_folder_path = os.path.join(main_folder_path, 'training_data')
plots_folder_path = os.path.join(main_folder_path, 'plots')
features_plots_folder_path = os.path.join(plots_folder_path,'features')

Mounted at /content/drive


## 5.2 Laden des Datensatzes

In [2]:
# Pfad zur gespeicherten Datei im 'processed_data' Ordner
file_path = os.path.join(processed_data_folder_path, 'wine_quality_processed.csv')

# Prüfen, ob die Datei existiert
if os.path.exists(file_path):
    print("File found! Loading...")
    data = pd.read_csv(file_path)
    print("Dataset loaded successfully.")
    print(f"Shape of the dataset: {data.shape}")
    display(data.head())
else:
    print(f"File not found at: {file_path}")


File found! Loading...
Dataset loaded successfully.
Shape of the dataset: (6497, 14)


,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_category
0,1,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,Okay
1,1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,Okay
2,1,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,Okay
3,1,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,Okay
4,1,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,Okay


## 5.3 Korrelationen analysieren

In [ ]:
# Funktion zur Erstellung und Speicherung der Korrelationsmatrix
def plot_and_save_correlation_matrix(data, target_variable, save_path=None):
    try:
        # Auswahl nur numerischer Spalten
        numerical_data = data.select_dtypes(include=['float64', 'int64'])

        # Korrelationsmatrix für numerische Spalten
        corr_matrix = numerical_data.corr()

        # Visualisierung der Korrelationsmatrix
        plt.figure(figsize=(12, 8))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
        plt.title("Korrelationsmatrix der numerischen Merkmale", fontsize=16)

        # Optional speichern
        if save_path:
            plt.savefig(save_path, bbox_inches='tight')

        plt.show()

        # Fokus auf die Zielvariable (z. B. 'quality')
        if target_variable in corr_matrix.columns:
            print(f"\nKorrelationen mit der Zielvariablen '{target_variable}':\n")
            print(corr_matrix[target_variable].sort_values(ascending=False))
        else:
            print(f"Zielvariable '{target_variable}' ist nicht in den numerischen Spalten enthalten.")
    except Exception as e:
        print(f"Fehler bei der Erstellung der Korrelationsmatrix: {e}")

# Beispielaufruf zum Speichern der Korrelationsmatrix
correlation_matrix_save_path = os.path.join(features_plots_folder_path, 'correlation_matrix.png')

plot_and_save_correlation_matrix(data, target_variable='quality', save_path=correlation_matrix_save_path)


## 5.4 Interpretation zwischen den Features und der Zielvariablen quality:

1) Stark korrelierte Features:
    - alcohol (0.444): Zeigt eine positive und moderate Korrelation mit der Qualität. Ein höherer Alkoholgehalt scheint tendenziell mit einer besseren Weinqualität verbunden zu sein.

2) Schwach positiv korrelierte Features:
    - type (0.119): Der Weintyp hat eine leichte, aber positive Beziehung zur Qualität. Dies könnte auf subtile Unterschiede zwischen Rot- und Weißweinen hinweisen.
    - citric acid (0.086), free sulfur dioxide (0.055), sulphates (0.038): Diese chemischen Eigenschaften haben eine sehr schwache positive Korrelation zur Qualität. Sie könnten indirekt durch andere Merkmale beeinflusst werden.

3) Nahezu unkorrelierte Features:
    - pH (0.019): Praktisch keine Korrelation mit der Weinqualität, daher vermutlich wenig informativ für das Modell.
    - residual sugar (-0.037) und total sulfur dioxide (-0.041): Sehr geringe Korrelationen, negativ und kaum relevant.
    
4) Schwach negativ korrelierte Features:
    - fixed acidity (-0.077): Eine leicht negative Beziehung zur Qualität, was auf eine möglicherweise geringe Bedeutung hindeutet.
    - chlorides (-0.201): Eine schwach negative Korrelation, was auf eine tendenziell schlechtere Qualität bei einem höheren Chloridgehalt hinweist.

5) Moderate negative Korrelationen:
    - volatile acidity (-0.266): Zeigt, dass höhere flüchtige Säurewerte mit einer schlechteren Qualität verbunden sind, was relevant sein könnte.
    - density (-0.306): Die Dichte weist eine moderate negative Korrelation auf, was bedeuten könnte, dass dichtere Weine tendenziell als weniger hochwertig empfunden werden.

6) Zusammenfassung:
    - Wichtige Features: alcohol, density, volatile acidity und möglicherweise chlorides sollten priorisiert werden, da sie die stärkste Korrelation mit der Zielvariablen aufweisen.
    - Weniger relevante Features: pH, residual sugar, total sulfur dioxide, und fixed acidity scheinen wenig oder keinen Einfluss auf die Weinqualität zu haben und könnten für die Modellierung weniger bedeutsam sein.

Diese Interpretation gibt eine Richtung vor, welche Features beim Modellieren priorisiert oder entfernt werden könnten. Jetzt entfernen wir die am wenigsten relevanten Features, um uns das weitere Vorgehen zu erleichtern. Diese Features sind: *pH*, *residual sugar*, und *total sulfur dioxide*.

## 5.5 Datensatz für Modelierung abspeichern

In [ ]:
# Output directory is already defined as 'model_folder_path'
output_file = "wine_quality.csv"  # Output file name

# Entfernen der Spalten
columns_to_remove = ['pH', 'residual sugar', 'total sulfur dioxide']
data = data.drop(columns=columns_to_remove, errors='ignore')
print(f"Spalten {columns_to_remove} wurden entfernt (falls vorhanden).")

# Datei im drive speichern
output_path = os.path.join(training_data_folder_path, output_file)  # Save to 'model' folder
data.to_csv(output_path, index=False)
print(f"Datei erfolgreich gespeichert unter '{output_path}'.")


Spalten ['pH', 'residual sugar', 'total sulfur dioxide'] wurden entfernt (falls vorhanden).
Datei erfolgreich gespeichert unter '/content/drive/MyDrive/wine_project/training_data/wine_quality.csv'.


## 5.6 Next Steps:
- Anwendung in Machine Learning: Entwicklung von Regressions- oder Klassifikationsmodellen,
Performance-Vergleich und –Interpretation
- Empfohlene Vorgehensweise:
    -Automatische Selektion: Wenn du dir unsicher bist, nutze Feature-Selection-Methoden (wie Lasso oder RFE), um Features basierend auf Daten und Modellanforderungen auszuwählen.

- Iterativer Ansatz:
    - Erste Modelliteration: Trainiere ein Modell mit allen Features, um die Basisleistung zu messen.
    - Optimierung: Entferne Features, die nachweislich keinen Beitrag leisten (z. B. durch geringe Korrelation oder niedrige Gewichtung im Modell).
    - Finales Modell: Nutze nur die wichtigsten Features.